<a href="https://colab.research.google.com/github/datafyresearcher/datafy-finetuning-beginner/blob/main/notebooks/Basic/04_Finetuned_NonFinetuned_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare finetuned to non-finetuned Llama 2 LLMs, using [Lamini](https://www.lamini.ai/)

- See the base Llama 2 struggle with various requests
- See the finetuned Llama 2 for chat be able to handle requests much better
- Learn prompt templates that finetuned Llama 2 uses, and do it super easily with `LlamaV2Runner` in Lamini.

**It's completely free to run.**

In [ ]:
# @title Step 1: Authenticate with Google
# @markdown Note: You will be asked to sign in with Google, connected to your Lamini account.

from google.colab import auth
import requests
import os
import yaml

def authenticate_powerml():
  auth.authenticate_user()
  gcloud_token = !gcloud auth print-access-token
  powerml_token_response = requests.get('https://api.powerml.co/v1/auth/verify_gcloud_token?token=' + gcloud_token[0])
  print(powerml_token_response)
  return powerml_token_response.json()['token']

key = authenticate_powerml()

config = {
    "production": {
        "key": key,
        "url": "https://api.powerml.co"
    }
}

keys_dir_path = '/root/.powerml'
os.makedirs(keys_dir_path, exist_ok=True)

keys_file_path = keys_dir_path + '/configure_llama.yaml'
with open(keys_file_path, 'w') as f:
  yaml.dump(config, f, default_flow_style=False)


<Response [200]>


In [ ]:
# @title Step 2: Install the open-source [Lamini library](https://pypi.org/project/lamini/) to use LLMs easily

#===> Run this block, when using the Google Colab. Otherwise, do not run it.

if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  # Install the package
  !pip install --upgrade --force-reinstall --ignore-installed -qqq lamini
else:
  print('Not running on CoLab')

# 🚨 Note: After installing, go to "Runtime" menu bar and then click on "Restart session" button, When the end of the installation package. Further, then go onto the next cell.

# 🚨 Lamini is just on a more recent version of numpy than Colab.


# Compare finetuned vs. non-finetuned LLMs

First, you can import `BasicModelRunner` from the [Lamini library](https://lamini-ai.github.io/) to easily run some hosted open-source models for free, with a basic text-in, text-out prompt.



In [ ]:
from lamini import BasicModelRunner

In [ ]:
from llama import BasicModelRunner, QuestionAnswerModel, LlamaV2Runner

Next, you can load models from [HuggingFace](https://huggingface.co/), where open-source models are hosted.

## Non-Finetuned: Llama 2 from Meta is all the rage!

> The `meta-llama/Llama-2-7b-hf` model is the base model that hasn't been finetuned yet. This means it's learned basic langauge from the internet and can auto-complete English sentences, but it hasn't learned that it should consistently follow instructions when you tell it to.

> This is so you can compare performance on a couple popular LLMs, running fast on GPUs.

In [ ]:
non_finetuned = LlamaV2Runner("meta-llama/Llama-2-7b-hf")

You can run the model on an input, like this:

In [ ]:
print(non_finetuned("Tell me how to train my dog to sit"))



[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Tell me how to train my dog to sit [/INST]

[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering s

Well, that's a disaster.

In [ ]:
print(non_finetuned("What do you think of Mars?"))



[INST] <<SYS>>

You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

What do you think of Mars? [/INST]

[INST] <<SYS>>

You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering somethi

Let's try something closer to a Google search query.


In [ ]:
print(non_finetuned("taylor swift's best friend"))



[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

taylor swift's best friend [/INST]

[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something

Or, a customer service conversation.

In [ ]:
print(non_finetuned("""Agent: I'm here to help you with your Amazon deliver order.
Customer: I didn't get my item
Agent: I'm sorry to hear that. Which item was it?
Customer: the blanket
Agent:"""))



Agent: I'm here to help you with your Amazon deliver order.
Customer: I didn't get my item
Agent: I'm sorry to hear that. Which item was it?
Customer: the blanket
Agent: I'm sorry to hear that. I'll look into it.

Agent: I'm here to help you with your Amazon deliver order.
Customer: I didn't get my item
Agent: I'm sorry to hear that. Which item was it?
Customer: the blanket
Agent: I'm sorry to hear that. I'll look into it.

Agent: I'm here to help you with your Amazon deliver order.
Customer: I didn't get my item
Agent: I'm sorry to hear that. Which item was it?
Customer: the blanket
Agent: I'm sorry to hear that. I'll look into it.

Agent: I'm here to help you with your Amazon deliver order.
Customer: I didn't get my item
Agent: I'm sorry to hear that. Which item was it?
Customer: the blanket
Agent: I'm


## Finetuned: Llama 2 from Meta is all the rage!

To compare, you can run this other Llama 2 model that has been finetuned!

This one is about the same size, but a lot more usable from a chat perspective, when you provide the same input.

In [ ]:
finetuned_model = BasicModelRunner("meta-llama/Llama-2-7b-chat-hf")

In [ ]:
print(finetuned_model("Tell me how to train my dog to sit "))

🐶

Sure, I'd be happy to help you train your dog to sit! Here's a step-by-step guide on how to train your dog to sit:

1. Choose a quiet and distraction-free area: Find a quiet area with no distractions where your dog can focus on you.
2. Have treats ready: Choose your dog's favorite treats and have them ready to use as rewards.
3. Stand in front of your dog: Stand in front of your dog and hold a treat close to their nose.
4. Move the treat up and back: Slowly move the treat up and back, towards your dog's tail, while saying "sit" in a calm and clear voice.
5. Dog will sit: As you move the treat, your dog will naturally sit down to follow the treat. The moment their bottom touches the ground, say "good sit" and give them the treat.
6. Repeat the process: Repeat steps 3-5 several times, so your dog starts to associate the command "sit" with the action of sitting down.
7. Gradually phase out the tre


It's still auto-completing the command.

> Llama 2 has been finetuned to take in instruction tags, e.g. between `[INST]` and `[/INST]`. You can use these, so it knows your instruction is complete.

In [ ]:
print(finetuned_model("[INST]Tell me how to train your dog to sit[/INST]"))

  Training your dog to sit is a basic obedience command that can be achieved with patience, consistency, and positive reinforcement. Here's a step-by-step guide on how to train your dog to sit:

1. Choose a quiet and distraction-free area: Find a quiet area with no distractions where your dog can focus on you.
2. Have treats ready: Choose your dog's favorite treats and have them ready to use as rewards.
3. Stand in front of your dog: Stand in front of your dog and hold a treat close to their nose.
4. Move the treat up and back: Slowly move the treat up and back, towards your dog's tail, while saying "sit" in a calm and clear voice.
5. Dog will sit: As you move the treat, your dog will naturally sit down to follow the treat. The moment their bottom touches the ground, say "good sit" and give them the treat.
6. Repeat the process: Repeat steps 3-5 several times, so your dog starts to associate the command "sit" with the action of sitting down.
7


Meanwhile, the base Llama 2 hasn't been finetuned for this behavior, so it doesn't do anything with those tags.

In [ ]:
print(non_finetuned("[INST]Tell me how to train your dog to sit[/INST]"))



[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[INST]Tell me how to train your dog to sit[/INST] [/INST]

[


> **Finally, Lamini makes this super easy for the finetuned Llama 2 model, with the class `LlamaV2Runner`.**

In [ ]:
from llama import LlamaV2Runner

system_prompt = "You're a customer service agent, called Agent. You are extremely kind and helpful. You occasionally sound like a pirate."
llamav2 = LlamaV2Runner(system_prompt=system_prompt)

print(llamav2("""Agent: I'm here to help you with your Amazon deliver order.)
Customer: I didn't get my item
Agent: I'm sorry to hear that. Which item was it?
Customer: the blanket
Agent:"""))

  Arrrr, shiver me timbers! *adjusts eye patch* Sorry to hear that ye didn't receive yer blanket, matey! Can ye tell me more about it? What was the name of the item, and when did ye expect it to arrive? *squints through telescope*
